In [ ]:
!pip install tensorflow==2.11.0 protobuf==3.19.6 numpy==1.23.5


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

In [9]:
# 1. Load and preprocess data
df = pd.read_csv("lungCancerDataset_balanced.csv")

In [10]:

# Encode target
df['LUNG_CANCER'] = df['LUNG_CANCER'].map({'YES': 1, 'NO': 0})

# Encode any non-numeric features
for col in df.columns:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

# Split features and target
X = df.drop("LUNG_CANCER", axis=1)
y = df["LUNG_CANCER"]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [12]:
# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tuning with early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
22/22 [==============================] - 1s 10ms/step - loss: 0.6901 - accuracy: 0.5745 - val_loss: 0.6081 - val_accuracy: 0.7225
Epoch 2/100
22/22 [==============================] - 0s 3ms/step - loss: 0.6161 - accuracy: 0.6889 - val_loss: 0.5344 - val_accuracy: 0.7746
Epoch 3/100
22/22 [==============================] - 0s 4ms/step - loss: 0.5599 - accuracy: 0.7670 - val_loss: 0.4771 - val_accuracy: 0.8092
Epoch 4/100
22/22 [==============================] - 0s 3ms/step - loss: 0.4976 - accuracy: 0.8017 - val_loss: 0.4367 - val_accuracy: 0.8266
Epoch 5/100
22/22 [==============================] - 0s 4ms/step - loss: 0.4736 - accuracy: 0.8177 - val_loss: 0.4036 - val_accuracy: 0.8266
Epoch 6/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4308 - accuracy: 0.8278 - val_loss: 0.3727 - val_accuracy: 0.8555
Epoch 7/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4028 - accuracy: 0.8423 - val_loss: 0.3511 - val_accuracy: 0.8728
Epoch 8/100


In [13]:
# Evaluate
loss, acc = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {acc:.4f}")

# Save the model
model.save("lung_cancer_model.keras")

7/7 [==============================] - 0s 1ms/step - loss: 0.2071 - accuracy: 0.9259
Test Accuracy: 0.9259
